##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [3]:
pip install tensorflow

     |████████████████████████████████| 195.6 MB 82 kB/s s eta 0:00:01          | 28.4 MB 4.0 MB/s eta 0:00:42
     |████████████████████████████████| 3.7 MB 38.0 MB/s eta 0:00:01
  Using cached typing_extensions-3.7.4.3-py3-none-any.whl (22 kB)
  Using cached absl_py-0.12.0-py3-none-any.whl (129 kB)
  Using cached keras_nightly-2.5.0.dev2021032900-py2.py3-none-any.whl (1.2 MB)
  Using cached wheel-0.36.2-py2.py3-none-any.whl (35 kB)
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached tensorflow_estimator-2.5.0-py2.py3-none-any.whl (462 kB)
  Using cached tensorboard-2.5.0-py3-none-any.whl (6.0 MB)
     |████████████████████████████████| 15.6 MB 19.8 MB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 46.8 MB/s eta 0:00:01
  Using cached wrapt-1.12.1.tar.gz (27 kB)
     |████████████████████████████████| 959 kB 21.7 MB/s eta 0:00:01
 

Note: you may need to restart the kernel to use updated packages.


# Text generation with an RNN

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/text/tutorials/text_generation"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/text/blob/master/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/text/blob/master/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/text/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This tutorial demonstrates how to generate text using a character-based RNN. You will work with a dataset of Shakespeare's writing from Andrej Karpathy's [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

Note: Enable GPU acceleration to execute this notebook faster. In Colab: *Runtime > Change runtime type > Hardware accelerator > GPU*.

This tutorial includes runnable code implemented using [tf.keras](https://www.tensorflow.org/guide/keras/sequential_model) and [eager execution](https://www.tensorflow.org/guide/eager). The following is the sample output when the model in this tutorial trained for 30 epochs, and started with the prompt "Q":

<pre>
QUEENE:
I had thought thou hadst a Roman; for the oracle,
Thus by All bids the man against the word,
Which are so weak of care, by old care done;
Your children were in your holy love,
And the precipitation through the bleeding throne.

BISHOP OF ELY:
Marry, and will, my lord, to weep in such a one were prettiest;
Yet now I was adopted heir
Of the world's lamentable day,
To watch the next way with his father with his face?

ESCALUS:
The cause why then we are all resolved more sons.

VOLUMNIA:
O, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, it is no sin it should be dead,
And love and pale as any will to that word.

QUEEN ELIZABETH:
But how long have I heard the soul for this world,
And show his hands of life be proved to stand.

PETRUCHIO:
I say he look'd on, if I must be content
To stay him from the fatal of our country's bliss.
His lordship pluck'd from this sentence then for prey,
And then let us twain, being the moon,
were she such a case as fills m
</pre>

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but consider:

* The model is character-based. When training started, the model did not know how to spell an English word, or that words were even a unit of text.

* The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.

* As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.

## Setup

### Import TensorFlow and other libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

### Download the Shakespeare dataset

Change the following line to run this code on your own data.

In [137]:
path_to_file = 'train2_RNN.txt'#tf.keras.utils.get_file('story.train.2.txt','https://storage.cloud.google.com/cs229data/story.train.2.txt')

### Read the data

First, look in the text:

In [138]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 17131 characters


In [140]:
# Take a look at the first 250 characters in text
print(text[:250])

This is Mudge.
He is Henry’s puppy.
Mudge wants Henry’s snack.
"No, Mudge," says Henry.
Mudge gets on Henry’s lap.
"No, Mudge," says Henry.
Mudge wants Henry’s snack.
Mudge gets on Henry’s head.
"No, Mudge," says Henry.
Mudge wants Henry’s snack.
Mud


In [141]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


## Process the text

### Vectorize the text

Before training, you need to convert the strings to a numerical representation. 

The `preprocessing.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [142]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Now create the `preprocessing.StringLookup` layer:

In [143]:
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab), mask_token=None)

It converts form tokens to character IDs:

In [144]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[38, 39, 40, 41, 42, 43, 44], [61, 62, 63]]>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use `preprocessing.StringLookup(..., invert=True)`.  

Note: Here instead of passing the original vocabulary generated with `sorted(set(text))` use the `get_vocabulary()` method of the `preprocessing.StringLookup` layer so that the `[UNK]` tokens is set the same way.

In [145]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

In [146]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

You can `tf.strings.reduce_join` to join the characters back into strings. 

In [147]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [148]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [149]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(17131,), dtype=int64, numpy=array([32, 45, 46, ...,  1,  1,  1])>

In [150]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [151]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

T
h
i
s
 
i
s
 
M
u


In [152]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

The `batch` method lets you easily convert these individual characters to sequences of the desired size.

In [153]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'T' b'h' b'i' b's' b' ' b'i' b's' b' ' b'M' b'u' b'd' b'g' b'e' b'.'
 b'\n' b'H' b'e' b' ' b'i' b's' b' ' b'H' b'e' b'n' b'r' b'y'
 b'\xe2\x80\x99' b's' b' ' b'p' b'u' b'p' b'p' b'y' b'.' b'\n' b'M' b'u'
 b'd' b'g' b'e' b' ' b'w' b'a' b'n' b't' b's' b' ' b'H' b'e' b'n' b'r'
 b'y' b'\xe2\x80\x99' b's' b' ' b's' b'n' b'a' b'c' b'k' b'.' b'\n' b'"'
 b'N' b'o' b',' b' ' b'M' b'u' b'd' b'g' b'e' b',' b'"' b' ' b's' b'a'
 b'y' b's' b' ' b'H' b'e' b'n' b'r' b'y' b'.' b'\n' b'M' b'u' b'd' b'g'
 b'e' b' ' b'g' b'e' b't' b's' b' ' b'o' b'n'], shape=(101,), dtype=string)


It's easier to see what this is doing if you join the tokens back into strings:

In [154]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'This is Mudge.\nHe is Henry\xe2\x80\x99s puppy.\nMudge wants Henry\xe2\x80\x99s snack.\n"No, Mudge," says Henry.\nMudge gets on'
b' Henry\xe2\x80\x99s lap.\n"No, Mudge," says Henry.\nMudge wants Henry\xe2\x80\x99s snack.\nMudge gets on Henry\xe2\x80\x99s head.\n"No, Mu'
b'dge," says Henry.\nMudge wants Henry\xe2\x80\x99s snack.\nMudge drools.\n"Aw, Mudge," says Henry.\nMudge looks cute.'
b'\nMudge looks very, very cute.\nMudge looks too cute.\n"Mudge, you are TOO cute," says Henry.\nHenry gets'
b' a snack for Mudge.\nIt is a CRACKER.\nMudge LOVES crackers.\nNow Henry has a snack.\nAnd Mudge has a sna'


For training you'll need a dataset of `(input, label)` pairs. Where `input` and 
`label` are sequences. At each time step the input is the current character and the label is the next character. 

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [155]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [156]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [157]:
dataset = sequences.map(split_input_target)

In [158]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'This is Mudge.\nHe is Henry\xe2\x80\x99s puppy.\nMudge wants Henry\xe2\x80\x99s snack.\n"No, Mudge," says Henry.\nMudge gets o'
Target: b'his is Mudge.\nHe is Henry\xe2\x80\x99s puppy.\nMudge wants Henry\xe2\x80\x99s snack.\n"No, Mudge," says Henry.\nMudge gets on'


### Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [159]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build The Model

This section defines the model as a `keras.Model` subclass (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)). 

This model has three layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [160]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [161]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [162]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](images/text_generation_training.png)

Note: For training you could use a `keras.Sequential` model here. To  generate text later you'll need to manage the RNN's internal state. It's simpler to include the state input and output options upfront, than it is to rearrange the model architecture later. For more details see the [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse).

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [163]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [164]:
model.summary()

Model: "my_model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      multiple                  16896     
_________________________________________________________________
gru_4 (GRU)                  multiple                  3938304   
_________________________________________________________________
dense_4 (Dense)              multiple                  67650     
Total params: 4,022,850
Trainable params: 4,022,850
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [165]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [166]:
sampled_indices

array([15,  7, 10, 25, 47, 44, 39, 64, 18,  4, 48, 45,  4, 48, 33, 46, 53,
        1,  3,  9, 44,  4, 25, 62, 52, 55, 29, 52, 29, 20,  1, 49, 48, 58,
       21,  2, 43, 40, 62, 32, 61, 42, 28, 22,  7,  1, 43, 52,  0, 32, 14,
       38, 32, 34, 30, 21, 10,  3, 41, 51,  6, 21, 34, 28, 47, 57, 49, 38,
       52, 40, 53, 34, 61, 51, 51, 28, 52, 54, 11, 55, 28, 31, 58, 28, 16,
       16, 65, 40, 56, 35, 16, 59, 14, 29,  6, 64, 33, 27, 20, 54])

Decode these to see the text predicted by this untrained model:

In [167]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b' The driver sheep forgets to steer.\nJeep in a heap Sheep weep.\nSheep sweep the heap.\nJeep for sale \xe2\x80\x94'

Next Char Predictions:
 b'C-2Mjgb\xe2\x80\x94F"kh"kUip\n!1g"MyorQoQH\nlkuI fcyTxePJ-\nfo[UNK]TBaTVRI2!dn,IVPjtlaocpVxnnPoq3rPSuPDD\xe2\x80\x99csWDvBQ,\xe2\x80\x94UOHq'


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.


In [168]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [169]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.1883135


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [170]:
tf.exp(mean_loss).numpy()

65.91154

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [171]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [172]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [173]:
EPOCHS = 200

In [174]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/200
2/2 [==============================] - 3s 1s/step - loss: 4.1682
Epoch 2/200
2/2 [==============================] - 2s 1s/step - loss: 3.9880
Epoch 3/200
2/2 [==============================] - 2s 1s/step - loss: 6.3579
Epoch 4/200
2/2 [==============================] - 2s 1s/step - loss: 3.9326
Epoch 5/200
2/2 [==============================] - 2s 1s/step - loss: 3.9850
Epoch 6/200
2/2 [==============================] - 2s 1s/step - loss: 3.9797
Epoch 7/200
2/2 [==============================] - 2s 1s/step - loss: 3.9562
Epoch 8/200
2/2 [==============================] - 2s 1s/step - loss: 3.9059
Epoch 9/200
2/2 [==============================] - 2s 1s/step - loss: 3.8290
Epoch 10/200
2/2 [==============================] - 2s 1s/step - loss: 3.7033
Epoch 11/200
2/2 [==============================] - 2s 1s/step - loss: 3.5144
Epoch 12/200
2/2 [==============================] - 2s 1s/step - loss: 3.4767
Epoch 13/200
2/2 [==============================] - 2s 1s/step - loss: 3.

2/2 [==============================] - 3s 1s/step - loss: 1.3755
Epoch 107/200
2/2 [==============================] - 3s 1s/step - loss: 1.3546
Epoch 108/200
2/2 [==============================] - 3s 1s/step - loss: 1.3385
Epoch 109/200
2/2 [==============================] - 3s 1s/step - loss: 1.3007
Epoch 110/200
2/2 [==============================] - 3s 1s/step - loss: 1.3390
Epoch 111/200
2/2 [==============================] - 3s 2s/step - loss: 1.2989
Epoch 112/200
2/2 [==============================] - 3s 1s/step - loss: 1.2678
Epoch 113/200
2/2 [==============================] - 3s 1s/step - loss: 1.2694
Epoch 114/200
2/2 [==============================] - 3s 1s/step - loss: 1.2245
Epoch 115/200
2/2 [==============================] - 3s 1s/step - loss: 1.2181
Epoch 116/200
2/2 [==============================] - 3s 1s/step - loss: 1.2187
Epoch 117/200
2/2 [==============================] - 3s 1s/step - loss: 1.1753
Epoch 118/200
2/2 [==============================] - 3s 1s/step - 

## Generate text

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.

![To generate text the model's output is fed back to the input](images/text_generation_sampling.png)

Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.


The following makes a single step prediction:

In [175]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [176]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [177]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:!
 Saik bat can carc.
Dinos cunting onr a wlith.
Then bex wants.
Bor wants to gan.
Cat wats the bag.
Sam ran up.
Pam ran down.
Dan ran up.
Dan ran down.
Dan Runce.
Some is Big to flace.
Dance!



The to ran egger.
One is big.
The rest are small.
Here is a tail.
Here is a tail.
Here is an ear.
Here is an ear.
Here is an ear.
Here is an eye.
Here is an eye.
Here is an eye.
Here is an eye.
Here is Big Bunny.
Here is a noge.
Har, wighor theep sweep.
Jeep for sawe — bakt and a kell and back.
Yes, we do!
 Come see!
 Momar babies learn to school.
I see the bus.
What is in my classroom?
 I see box.
"Is it a dog egg?
" "No," says Cat.
"Is it a cow egg?
" "YE, Sam!



I am Tim.
My twin is Jim.
I see a cat.
I see a car.



I see a cat.
I se a dog.
A bug in a rug.
A bug in the grass.
ACkular baby watt.
"I's wancing Pila a cat hike my car door ug?
 Writing?
 I like to read.
Do you goes that’s st, pilll!
 Thim poll to see me wars.
Big Cat can dawe.
"See mit salf.
See Otto go left comed.
The ba

The easiest thing you can do to improve the results is to train it for longer (try `EPOCHS = 30`).

You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

If you want the model to generate text *faster* the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above. 

In [178]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b'ROMEO:.\n Cancin thim in hime fore.\nHirk is a foot.\nHere is a tail.\nHeris in a jug.\nRinos jump, in ear.\nHere is an ear.\nHere is an ear.\nHere is an ear.\nHere is an ear.\nHere is an ear.\n\n\n\nI go home.\nI see eggs.\nI like you.\nI can see.\nI can run.\nI can hide.\nI can side.\nI can pep.\nI can ride.\nMom can jump too.\nI can can jump.\nA caw rin tome curery looked and home and hos.\nI see a squirrel.\n\n\n\nI goe the ball to me and mrower sheng down, then stoes a has a bet.\nI see thet chme sad.\nGree go to some lett comes out.\nSheep shove.\nSheep gru.\nKnock, Dog.\nDown choued the ice.\nThis polar baby loves to swim.\nThis polar baby lokes to fish.\nThis polar baby loves to sleep!\n Oh, no!\n Oh, no!\n Oh, no!\n Oh, no!\n It hits a carr.\nI he mam.\nI tow the ball.\nJam I see a hole and a flag and a necklace and a sword.\nA box can be many things," I say.\nThe box is a cage.\n"I am out!\n" he said.\n"Me too\',\' said the Chick.\n"I tous.\nI se home.\n\n\n\nDr

## Export the generator

This single-step model can easily be [saved and restored](https://www.tensorflow.org/guide/saved_model), allowing you to use it anywhere a `tf.saved_model` is accepted.

In [179]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')


FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.


INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


In [181]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:p.
Sheep ghar sits burthrf.
On, no!
 Dog can flip fraping into the cat.
I like my shirs.
I like megp.
Jack has a cap.
No way jusping 1, 2," 3.
Dinos peaking enca.
Dinos peeking the till.
There was dirt everywhere!
 On Friday.
Cookie ran into the clost.
There wer a thell seen looked for food.
"No at dancing to stwe lithoroure.
See Otto go tleep!
 Jeeping, laging all the way.
Dinos justing 1, 2, 3.
Dinos pasting red and bl eg.
She fring eats for a bug.
They all went to hump.
Ree can jump too.
I can climb.
Mom can climb too.
I can run.
Mom can run too.
I can sit.
I can peek.
I han tak.
I am loxt.
Cat Igg to to cleep!
 Oh, no!
 Oh, no!
 It hits a car.
I like my gare.
I din and my cat?
 This is not my egg can cate.
" The warther dings!
 The bag rox my big.
I hus my met.
I hug my bear.
I hug my mide.
I lo tue.
I see a butt.
I saw a hes a snack.
And CiAkbou a cate.
I see a hou.
I like my puntrif.
I am Sam.
I can side.
I can peek.



Max cannot ride a chicken.
But the fish can eat.
The f

## Advanced: Customized Training

The above training procedure is simple, but does not give you much control.
It uses teacher-forcing which prevents bad predictions from being fed back to the model, so the model never learns to recover from mistakes.

So now that you've seen how to run the model manually next you'll implement the training loop. This gives a starting point if, for example, you want to implement _curriculum  learning_ to help stabilize the model's open-loop output.

The most important part of a custom training loop is the train step function.

Use `tf.GradientTape` to track the gradients. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The basic procedure is:

1. Execute the model and calculate the loss under a `tf.GradientTape`.
2. Calculate the updates and apply them to the model using the optimizer.

In [182]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

The above implementation of the `train_step` method follows [Keras' `train_step` conventions](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit). This is optional, but it allows you to change the behavior of the train step and still use keras' `Model.compile` and `Model.fit` methods.

In [183]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [184]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [186]:
model.fit(dataset, epochs=100)

Epoch 1/100
2/2 [==============================] - 3s 1s/step - loss: 3.7106
Epoch 2/100
2/2 [==============================] - 3s 1s/step - loss: 3.4578
Epoch 3/100
2/2 [==============================] - 3s 1s/step - loss: 3.3174
Epoch 4/100
2/2 [==============================] - 3s 2s/step - loss: 3.2414
Epoch 5/100
2/2 [==============================] - 3s 1s/step - loss: 3.1723
Epoch 6/100
2/2 [==============================] - 3s 1s/step - loss: 3.1231
Epoch 7/100
2/2 [==============================] - 3s 1s/step - loss: 3.1023
Epoch 8/100
2/2 [==============================] - 3s 1s/step - loss: 3.0549
Epoch 9/100
2/2 [==============================] - 3s 1s/step - loss: 3.0045
Epoch 10/100
2/2 [==============================] - 3s 2s/step - loss: 2.9523
Epoch 11/100
2/2 [==============================] - 3s 1s/step - loss: 2.9117
Epoch 12/100
2/2 [==============================] - 3s 2s/step - loss: 2.8551
Epoch 13/100
2/2 [==============================] - 4s 2s/step - loss: 2.

Or if you need more control, you can write your own complete custom training loop:

In [187]:
EPOCHS = 50

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 1.4276

Epoch 1 Loss: 1.3459
Time taken for 1 epoch 3.01 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.3278

Epoch 2 Loss: 1.3230
Time taken for 1 epoch 2.30 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.3659

Epoch 3 Loss: 1.3235
Time taken for 1 epoch 2.17 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.2336

Epoch 4 Loss: 1.3027
Time taken for 1 epoch 3.35 sec
________________________________________________________________________________
Epoch 5 Batch 0 Loss 1.2744

Epoch 5 Loss: 1.2607
Time taken for 1 epoch 3.51 sec
________________________________________________________________________________
Epoch 6 Batch 0 Loss 1.2686

Epoch 6 Loss: 1.2386
Time taken for 1 epoch 2.42 sec
________________________________________________________________________________
Epoch 7 Batch 0 Loss 1

In [188]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:.
Y.
Dance driins in the s andows.
Jimp is a fack.
He put is wirm.
Max pits o fise mook.
Dinos dancing in a booke.



Herm ho seee my cat?
 This is not my cat!
 Have you seen my cat?
 This is not my cat!
 Have you seen my cat?
 This is not my cat!
 Have you seen my cat?
 This is not my cat jump toe.
I see boot.
I see the flag.
I sue the chille.
The pest are small.
Hen loves them all.



I like stars.
Yerrow shark a frignt, berr.
Jeep in a heap Sheep weep.
Sheep sweep the heap.
Jeep for sale — see a squirrel.
I see mice.
I see a bute.
I way We wam?



I am a fug.
I am a bear.
"Stay out!
" I growl.
I's buck.
 I made a snack.
I hug my read.
I hug my cat.
I hug my drack.
I like my bike.



Max likes to rand books.
Max checks out him home carkoot.
He the box is back.
Jilp a nowe.
See Otto go to fleer.
Fox goes under.
Go, Fox!
 Go!
 Fox gets flowets.
Jell in a hit.
This polar baby loves to sleep!
 Oh, no!
 Oh, no!
 It hits a car.
I like my pan.
I like my truck.
I like my truck.
I like 